In [2]:
import os
import sys
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
from dotenv import load_dotenv


# DB connection
try:
    load_dotenv()
    SQLUID = os.getenv("SQLUID")
    SQLPASSWORD = os.getenv("SQLPASSWORD")
    DRIVER = "ODBC Driver 18 for SQL Server"
    SQLSERVER = "localhost"
    SQLDB = "ContosoRetailDW"
    engine_stmt = f"mssql+pyodbc://{SQLUID}:{SQLPASSWORD}@{SQLSERVER}/{SQLDB}?TrustServerCertificate=yes&driver={DRIVER}"
    engine = create_engine(engine_stmt)
    print("DB connection established.")
except SQLAlchemyError as e:
    print("Error: DB connection failed!", e.__cause__, sep="\n")
    sys.exit()

DB connection established.


In [21]:
sql_query = "SELECT * FROM FactSales"
chunksize = 10000
connection = engine.connect().execution_options(stream_results=True, max_row_buffer=chunksize)
df = pd.read_sql(sql_query, connection, chunksize=chunksize)
categories = ["ProductKey", "SalesAmount"]
output = pd.DataFrame()
for chunk in df:
    df_cat = chunk[categories]
    summary = df_cat.groupby(["ProductKey"])["SalesAmount"].sum().reset_index()
    output = pd.concat([output, summary], ignore_index=True)

In [23]:
total_summary = output.groupby(["ProductKey"])["SalesAmount"].sum().reset_index()
total_summary

,ProductKey,SalesAmount
0,1,2.551357e+05
1,2,2.441620e+05
2,3,2.857652e+05
3,4,4.104208e+05
4,5,4.166276e+05
...,...,...
2511,2513,8.157691e+06
2512,2514,7.974490e+06
2513,2515,1.304813e+06
2514,2516,1.391502e+06
